## 01-BIBLIOTECAS

In [1]:
!pip install pandas numpy scikit-learn xgboost shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 7.8 MB/s eta 0:00:00


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from io import StringIO
import requests
from sklearn.pipeline import Pipeline
from joblib import dump
import sklearn
from io import StringIO
from io import BytesIO
from joblib import load
import xgboost as xgb
import shap

In [3]:
pip install --upgrade pandas numpy scikit-learn requests shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.25.0 req

##02-CARGA DE PREPROCESADOR Y MODELO.
Lo tenemos que hacer de esta forma, porque usando .joblib, se cargan dañados.

In [3]:
# Función para filtrar columnas
def filtrar_columnas(df):
    columnas_a_mantener = [
        'International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls'                   # Como se puede observar, en el prepro, NO se incluye el target
    ]
    return df[columnas_a_mantener]

# Función que transforma 'International plan' a int.
def mapear_international_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['International plan'] = df_copy['International plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función que transforma 'Voice mail plan' a int.
def mapear_voice_mail_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['Voice mail plan'] = df_copy['Voice mail plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función para eliminar outliers
def eliminar_outliers(df):
    columns_to_check = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()

    for column in columns_to_check:
        q1 = df_copy[column].quantile(0.25)
        q3 = df_copy[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Eliminar las filas con outliers solo en la columna actual
        df_copy = df_copy.drop(df_copy[(df_copy[column] < lower_bound) | (df_copy[column] > upper_bound)].index)

    return df_copy





# Creación de transformadores
Best_caracter = FunctionTransformer(filtrar_columnas)
Mapeo_01 = FunctionTransformer(mapear_international_plan)
Mapeo_02 = FunctionTransformer(mapear_voice_mail_plan)
Outliers_drop = FunctionTransformer(eliminar_outliers)


# Creación del pipeline de preprocesamiento
Prepro01 = Pipeline(steps=[
    ("Best_features", Best_caracter),
    ("Mapeo_01", Mapeo_01),
    ("Mapeo_02", Mapeo_02),
    ("Outliers_drop", Outliers_drop),
])
Prepro01

Pipeline(steps=[('Best_features',
                 FunctionTransformer(func=<function filtrar_columnas at 0x7b4b0c092e60>)),
                ('Mapeo_01',
                 FunctionTransformer(func=<function mapear_international_plan at 0x7b4b0c092cb0>)),
                ('Mapeo_02',
                 FunctionTransformer(func=<function mapear_voice_mail_plan at 0x7b4aa7862170>)),
                ('Outliers_drop',
                 FunctionTransformer(func=<function eliminar_outliers at 0x7b4aa7862200>))])

In [4]:
# Función para filtrar columnas
def filtrar_columnas(df):
    columnas_a_mantener = [
        'International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls', 'Churn'                  # Como se puede observar, en el prepro, NO se incluye el target
    ]
    return df[columnas_a_mantener]

# Función que transforma 'International plan' a int.
def mapear_international_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['International plan'] = df_copy['International plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función que transforma 'Voice mail plan' a int.
def mapear_voice_mail_plan(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['Voice mail plan'] = df_copy['Voice mail plan'].map({'Yes': 1, 'No': 0})
    return df_copy

# Función para eliminar outliers
def eliminar_outliers(df):
    columns_to_check = ['Number vmail messages', 'Total day minutes',
                             'Total day charge', 'Total eve minutes',
                             'Total eve charge', 'Total night minutes',
                             'Total night charge', 'Total intl minutes',
                             'Total intl calls']

    df_copy = df.copy()

    for column in columns_to_check:
        q1 = df_copy[column].quantile(0.25)
        q3 = df_copy[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Eliminar las filas con outliers solo en la columna actual
        df_copy = df_copy.drop(df_copy[(df_copy[column] < lower_bound) | (df_copy[column] > upper_bound)].index)

    return df_copy



# Definir función de mapeo de la variable objetivo
def mapear_target(df):
    df_copy = df.copy()  # Crear una copia del DataFrame
    df_copy['Churn'] = df_copy['Churn'].astype(int)  # Convertir valores booleanos a enteros (0 para False, 1 para True)
    return df_copy

# Df virgen
file_id = "16ypxCIBr9wSGVEaXqWdZUfz9w4xzccwo"
download_link = f"https://drive.google.com/uc?id={file_id}"
response = requests.get(download_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

#Aplicamos funciones
df1 = filtrar_columnas(df)
df1 = mapear_international_plan(df1)
df1 = mapear_voice_mail_plan(df1)
df1 = eliminar_outliers(df1)
df1 = mapear_target(df1)


# Definimos las características según lo ya definido en la etapa de elección de modelo (no incluido en el Prepro01).
X = df1[['International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls']]
y = df1.Churn

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Entrenar el modelo XGBoost
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

RandomForestClassifier()

In [5]:
# IMPORTAMOS SET VIRGEN
file_id = "16ypxCIBr9wSGVEaXqWdZUfz9w4xzccwo"
download_link = f"https://drive.google.com/uc?id={file_id}"
response = requests.get(download_link)
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, encoding='utf-8')

In [6]:
X = df[['International plan', 'Voice mail plan', 'Number vmail messages',
        'Total day minutes', 'Total day charge', 'Total eve minutes',
        'Total eve charge', 'Total night minutes', 'Total night charge',
        'Total intl minutes', 'Total intl calls']]
y = df.Churn

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
X_test=Prepro01.fit_transform(X_test)
X_train=Prepro01.fit_transform(X_train)

In [8]:
dd=modelo.predict(X_test)
dd

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## 03-SHAP

In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2002 entries, 1855 to 860
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   International plan     2002 non-null   int64  
 1   Voice mail plan        2002 non-null   int64  
 2   Number vmail messages  2002 non-null   int64  
 3   Total day minutes      2002 non-null   float64
 4   Total day charge       2002 non-null   float64
 5   Total eve minutes      2002 non-null   float64
 6   Total eve charge       2002 non-null   float64
 7   Total night minutes    2002 non-null   float64
 8   Total night charge     2002 non-null   float64
 9   Total intl minutes     2002 non-null   float64
 10  Total intl calls       2002 non-null   int64  
dtypes: float64(7), int64(4)
memory usage: 187.7 KB


In [26]:
X_train

,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day charge,Total eve minutes,Total eve charge,Total night minutes,Total night charge,Total intl minutes,Total intl calls
1855,0,0,0,159.1,27.05,202.5,17.21,233.1,10.49,11.5,6
2244,0,0,0,87.7,14.91,214.8,18.26,201.3,9.06,10.8,6
2525,0,0,0,138.1,23.48,158.2,13.45,215.7,9.71,10.3,2
961,0,0,0,134.3,22.83,202.3,17.20,195.9,8.82,12.6,5
1618,0,0,0,145.0,24.65,209.1,17.77,328.5,14.78,14.6,2
...,...,...,...,...,...,...,...,...,...,...,...
1638,0,0,0,215.3,36.60,242.4,20.60,279.8,12.59,12.1,9
1095,0,0,0,115.7,19.67,127.8,10.86,107.5,4.84,9.3,6
1130,0,0,0,203.2,34.54,235.8,20.04,224.9,10.12,15.1,6
1294,0,0,0,226.9,38.57,201.6,17.14,130.2,5.86,13.2,5


In [27]:
# Crear el objeto explainer de SHAP
explainer = shap.Explainer(modelo,X_train)

In [28]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 472 entries, 1915 to 1992
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   International plan     472 non-null    int64  
 1   Voice mail plan        472 non-null    int64  
 2   Number vmail messages  472 non-null    int64  
 3   Total day minutes      472 non-null    float64
 4   Total day charge       472 non-null    float64
 5   Total eve minutes      472 non-null    float64
 6   Total eve charge       472 non-null    float64
 7   Total night minutes    472 non-null    float64
 8   Total night charge     472 non-null    float64
 9   Total intl minutes     472 non-null    float64
 10  Total intl calls       472 non-null    int64  
dtypes: float64(7), int64(4)
memory usage: 44.2 KB


In [29]:
shap_values = explainer(X_test)

 93%|=================== | 882/944 [00:17<00:01]       

ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. This check failed because for one of the samples the sum of the SHAP values was 0.839400, while the model output was 0.820000. If this difference is acceptable you can set check_additivity=False to disable this check.

In [30]:
print("Forma de shap_values:", shap_values.shape)
print("Forma de los datos de entrada:", X_test.shape)
print("Primeros elementos de shap_values:", shap_values[:5])


NameError: name 'shap_values' is not defined

In [17]:
shap.plots.bar(shap_values)

NameError: name 'shap_values' is not defined

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.scatter(shap_values[:,"Total day minutes"],color=shap_values)

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0],figsize=(15,3),link='logit')

## 04-CONCLUSIONES